In [1]:
import numpy as np

In [2]:
import dill

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
from sklearn.metrics import confusion_matrix

In [5]:
from random import sample

In [6]:
def predict_hand(probs, t):
    return np.array([roundt(a[0][1], t) for a in probs])

In [7]:
def roundt(x, t):
    return 1 if x > t else 0

In [8]:
def get_metrics(model, X, Y, t):
    all_probs = model.predict_proba(X)
    accs, precs, recs = [], [], []
    for i,y in enumerate(Y):
        probs = np.array(all_probs)[:,:,1][:,i]
        preds = [roundt(p, t) for p in probs]
        tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
        acc, prec, rec = (tp+tn)/(tn+fp+fn+tp), tp/(tp+fp), tp/(tp+fn)
        accs.append(acc)
        precs.append(prec)
        recs.append(rec)
    return f"acc: {np.mean(accs):.4f}, prec: {np.mean(precs):.4f}, rec: {np.mean(recs):.4f}, sds:  {np.std(accs):.4f}, {np.std(precs):.4f}, {np.std(recs):.4f}"
    

In [9]:
def dill_save(model, file_name):
    with open(file_name, "wb") as f:
        dill.dump(model, f)
def dill_load(file_name):
    return dill.load(open(file_name, "rb"))

In [10]:
states = np.load('states2.npy')
hands = np.load('hands.npy')

In [11]:
states2d = states.reshape(2571176, 4*52)

In [12]:
states.shape

(2571176, 4, 52)

In [14]:
states3 = np.zeros((2571176, 3, 52))

In [15]:
states3[:,0,:] = states[:,0,:] + states[:,2,:]
states3[:,1,:] = states[:,1,:]
states3[:,2,:] = states[:,3,:]
states32d = states3.reshape(2571176, 3*52)

In [16]:
sum_states2d = np.sum(states32d, axis=1)

In [17]:
max(sum_states2d), min(sum_states2d), np.mean(sum_states2d), np.median(sum_states2d), sum_states2d[:80]

(44.0,
 12.0,
 18.437320510147885,
 18.0,
 array([12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 24.,
        12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 24.,
        12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 12.,
        12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 24., 24.,
        12., 12., 14., 14., 16., 16., 18., 18., 20., 20., 22., 12., 12.,
        14., 14., 16., 16., 18., 18., 20., 20., 22., 22., 12., 12., 14.,
        14., 16.]))

# basic

In [62]:
rf0 = RandomForestClassifier()

In [40]:
rf10K = RandomForestClassifier()
rf10K.fit(states2d[:10000], hands[:10000])

RandomForestClassifier()

In [63]:
rf0.fit(states2d[:100000], hands[:100000])

RandomForestClassifier()

In [90]:
rf0.predict_proba(states2d[100000:100001])[:3]

[array([[0.97, 0.03]]), array([[0.25, 0.75]]), array([[0.2, 0.8]])]

In [99]:
np.array(rf0.predict_proba(states2d[100000:100002]))[:,:,1][:,0]

array([0.03, 0.75, 0.8 , 0.09, 0.09, 0.27, 0.04, 0.06, 0.07, 0.03, 0.04,
       0.03, 0.04, 0.8 , 0.58, 0.09, 0.78, 0.1 , 0.02, 0.08, 0.8 , 0.77,
       0.73, 0.02, 0.04, 0.05, 0.08, 0.05, 0.1 , 0.76, 0.1 , 0.08, 0.03,
       0.03, 0.06, 0.01, 0.05, 0.05, 0.02, 0.12, 0.78, 0.03, 0.04, 0.04,
       0.08, 0.02, 0.11, 0.07, 0.03, 0.02, 0.01, 0.03])

In [ ]:
rf0.predict_proba(states2d[100000:100002])
# [array([[0.97, 0.03],
#         [0.23, 0.77]]), array([[0.25, 0.75],
#         [0.96, 0.04]]), array([[0.2 , 0.8 ],
#         [0.98, 0.02]]), array([[0.91, 0.09],
#         [0.89, 0.11]]), array([[0.91, 0.09],

In [71]:
predict_hand(rf0.predict_proba(states2d[100001:100002]), 0.3)

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0])

In [84]:
hands[1000000:1000001]-predict_hand(rf0.predict_proba(states2d[1000000:1000001]), 0.3)

array([[ 1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  0.,  0.,  1.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
        -1.,  1., -1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1., -1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [49]:
np.array([roundt(a[0][1], 0.3) for a in rf0.predict_proba(tempX[10000:10001])])

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0])

In [52]:
confusion_matrix(tempY[10000], np.array([roundt(a[0][1], 0.3) for a in rf0.predict_proba(tempX[10000:10001])]))

array([[33,  9],
       [ 5,  5]])

In [55]:
confusion_matrix(tempY[10001], np.array([roundt(a[0][1], 0.3) for a in rf0.predict_proba(tempX[10001:10002])]))

array([[37,  5],
       [ 3,  7]])

In [57]:
confusion_matrix(tempY[10002], np.array([roundt(a[0][1], 0.25) for a in rf0.predict_proba(tempX[10002:10003])]))

array([[31, 11],
       [ 2,  8]])

In [114]:
accs, precs, recs = get_metrics(rf0, states2d[100000:101000], hands[100000:101000], 0.3)

In [115]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.7914038461538462, 0.4351283355533355, 0.2911)

In [116]:
accs, precs, recs = get_metrics(rf0, states2d[100000:101000], hands[100000:101000], 0.25)

In [117]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.7448076923076924, 0.38473894521800867, 0.5337000000000001)

In [80]:
accs, precs, recs = get_metrics(model, states2d[200000:201000], hands[200000:201000], 0.25)

In [81]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.7293846153846154, 0.30772695291035074, 0.3482)

In [82]:
accs, precs, recs = get_metrics(model, states2d[300000:301000], hands[300000:301000], 0.25)

In [83]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.744326923076923, 0.3847426527419949, 0.5647000000000001)

In [46]:
accs, precs, recs = get_metrics(rf10K, states2d[100000:101000], hands[100000:101000], 0.25)

In [47]:
np.mean(accs), np.mean(precs), np.mean(recs)

(0.7383461538461538, 0.35371525463270814, 0.4561)

# save

In [124]:
with open("rf.obj","wb") as f:
    dill.dump(rf0,f)

In [8]:
rf = dill.load(open("rf.obj","rb"))

# predict hand methods

In [18]:
# state is array of length 208 made from flattened 4x52 array where 
# row 0 is past discards (cards opponent discarded)
# row 1 is known cards in opponent's hand (picked up by opponent and not discarded)
# row 2 is past faceUpCards not picked up by opponent
# row 3 is your own hand
def predict_opponent_hand(rf, state):
    probs = rf.predict_proba([state])
    return np.array(probs)[:,:,1][:,0]

In [19]:
def predict_cards_t(hand_probs, t):
    return np.array([roundt(p, t) for p in hand_probs])

In [37]:
def predict_cards_max(hand_probs):
    inds = hand_probs.argsort()[-10:][::-1]
    ret = np.zeros((52))
    ret[inds] = 1
    return ret.astype(int)

In [36]:
states2d[100000].shape

(208,)

In [14]:
predict_opponent_hand(model, states2d[100000])

array([0.03, 0.75, 0.8 , 0.09, 0.09, 0.27, 0.04, 0.06, 0.07, 0.03, 0.04,
       0.03, 0.04, 0.8 , 0.58, 0.09, 0.78, 0.1 , 0.02, 0.08, 0.8 , 0.77,
       0.73, 0.02, 0.04, 0.05, 0.08, 0.05, 0.1 , 0.76, 0.1 , 0.08, 0.03,
       0.03, 0.06, 0.01, 0.05, 0.05, 0.02, 0.12, 0.78, 0.03, 0.04, 0.04,
       0.08, 0.02, 0.11, 0.07, 0.03, 0.02, 0.01, 0.03])

In [26]:
predict_cards_t(predict_opponent_hand(model, states2d[100000]), 0.3)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
predict_cards_max(predict_opponent_hand(model, states2d[100000]))

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

# compare to random

In [26]:
from random import sample

In [27]:
def predict_random(X):
    known_opponent = X[1]
    known = X[0] + X[1] + X[2] + X[3]
    n_guess = 10 - sum(X[1])
    inds = np.where(known == 0)
    known_opponent[sample(list(inds[0]), int(n_guess))] = 1
    return known_opponent

In [61]:
states[100000][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [74]:
predict_random(states[100000])

[ 1  2  3  4  7  8 10 13 14 15 16 17 18 20 22 23 24 25 26 27 28 29 30 35
 36 37 38 39 40 45 47 51] 9


array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0.])

In [25]:
def get_metrics_rand(X, Y):
    accs, precs, recs = [], [], []
    for i,y in enumerate(Y):
        preds = predict_random(X[i])
        tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
        acc, prec, rec = (tp+tn)/(tn+fp+fn+tp), tp/(tp+fp), tp/(tp+fn)
        accs.append(acc)
        precs.append(prec)
        recs.append(rec)
    return np.mean(accs), np.mean(precs), np.mean(recs)

In [77]:
get_metrics_rand(states[100000:101000], hands[100000:101000])

(0.7439230769230769, 0.3342, 0.3342)

In [78]:
get_metrics_rand(states[200000:201000], hands[200000:201000])

(0.7397307692307693, 0.3233, 0.3233)

In [87]:
get_metrics_rand(states[300000:301000], hands[300000:301000])

(0.7403076923076922, 0.32480000000000003, 0.32480000000000003)

# sample_weight

In [94]:
states2d[:2].sum(axis=1)

array([12., 12.])

In [95]:
rfw = RandomForestClassifier()
rfw.fit(states32d[:100000], hands[:100000], sample_weight=states32d[:100000].sum(axis=1))

RandomForestClassifier()

In [97]:
get_metrics(rfw, states2d[100000:101000], hands[100000:101000], 0.25)

(0.7351346153846154, 0.3255840747101196, 0.37429999999999997)

In [99]:
get_metrics(rfw, states2d[200000:201000], hands[200000:201000], 0.25)

(0.7325, 0.31833686934410615, 0.37089999999999995)

In [101]:
get_metrics(rfw, states2d[300000:301000], hands[300000:301000], 0.25)

(0.7333461538461538, 0.32423115121033075, 0.3871)

# state 3x52

In [17]:
rf3 = RandomForestClassifier()
rf3.fit(states32d[:100000], hands[:100000])

RandomForestClassifier()

In [18]:
get_metrics(rf3, states32d[100000:101000], hands[100000:101000], 0.25)

(0.7433653846153846, 0.38316873654381395, 0.5579)

In [21]:
with open("rf3.obj","wb") as f:
    dill.dump(rf3,f)

In [82]:
rf3 = dill.load(open("rf3.obj","rb"))

In [107]:
get_metrics(rf3, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3, states32d[300000:301000], hands[300000:301000], 0.25)

('acc: 0.7434, prec: 0.3832, rec: 0.5579, sds:  0.0668, 0.1232, 0.2135',
 'acc: 0.7377, prec: 0.3728, rec: 0.5572, sds:  0.0626, 0.1174, 0.2129',
 'acc: 0.7465, prec: 0.3893, rec: 0.5882, sds:  0.0639, 0.1212, 0.2217')

In [92]:
predict_opponent_hand(rf3, states32d[100016])

array([0.32, 0.2 , 0.18, 0.35, 0.25, 0.19, 0.22, 0.22, 0.21, 0.12, 0.07,
       0.14, 0.13, 0.16, 0.24, 0.08, 0.15, 0.3 , 0.19, 0.13, 0.3 , 0.2 ,
       0.16, 0.28, 0.35, 0.21, 0.26, 0.2 , 0.32, 0.39, 0.17, 0.26, 0.15,
       0.22, 0.14, 0.08, 0.1 , 0.16, 0.16, 0.1 , 0.27, 0.09, 0.36, 0.18,
       0.23, 0.22, 0.12, 0.14, 0.03, 0.05, 0.11, 0.14])

In [91]:
sum(states32d[100016])

18.0

In [93]:
hands[100016]

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0.])

In [22]:
rf310K = RandomForestClassifier()
rf310K.fit(states32d[:10000], hands[:10000])

RandomForestClassifier()

In [23]:
get_metrics(rf310K, states32d[100000:101000], hands[100000:101000], 0.25)

(0.7369038461538462, 0.35614782810928325, 0.4717)

# Different models

In [ ]:
# KNN, ExtraTrees

In [1]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
et3 = ExtraTreesClassifier()

In [14]:
et3.fit(states32d[:100000], hands[:100000])

ExtraTreesClassifier()

In [15]:
get_metrics(et3, states32d[100000:101000], hands[100000:101000], 0.25)

(0.7413461538461538, 0.37895119526306087, 0.5532999999999999)

In [16]:
get_metrics(et3, states32d[200000:201000], hands[200000:201000], 0.25)

(0.7416730769230769, 0.38089193878446975, 0.5619000000000001)

In [17]:
get_metrics(et3, states32d[300000:301000], hands[300000:301000], 0.25)

(0.7432692307692308, 0.3847448814386508, 0.5821)

In [18]:
knn3 = KNeighborsClassifier()
knn3.fit(states32d[:100000], hands[:100000])

KNeighborsClassifier()

In [19]:
get_metrics(knn3, states32d[100000:101000], hands[100000:101000], 0.25)

(0.6908846153846153, 0.28159246576436975, 0.3926)

In [20]:
get_metrics(knn3, states32d[200000:201000], hands[200000:201000], 0.25)

(0.6910192307692307, 0.2809286130836828, 0.3891)

# Pruning Trees

In [ ]:
# rf3
# (0.7433653846153846, 0.38316873654381395, 0.5579)
# (0.7377307692307693, 0.37279639612623355, 0.5572)
# (0.7465192307692308, 0.3892681196049281, 0.5882000000000001)

In [12]:
rf3p01 = RandomForestClassifier(ccp_alpha=.01)
rf3p01.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(ccp_alpha=0.01)

In [17]:
get_metrics(rf3p01, states32d[100000:101000], hands[100000:101000], 0.26)

(0.677576923076923, 0.2583571428571429, 0.3617)

In [14]:
get_metrics(rf3p01, states32d[200000:201000], hands[200000:201000], 0.25)

(0.6560384615384617, 0.2680588235294118, 0.4557)

In [19]:
rf3p4 = RandomForestClassifier(min_samples_split=4)
rf3p4.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_split=4)

In [22]:
get_metrics(rf3p4, states32d[100000:101000], hands[100000:101000], 0.255)

(0.7412115384615384, 0.3811134008518515, 0.5574000000000001)

In [23]:
get_metrics(rf3p4, states32d[200000:201000], hands[200000:201000], 0.255)

(0.7395961538461538, 0.379672770366888, 0.5700999999999999)

In [29]:
get_metrics(rf3p4, states32d[300000:301000], hands[300000:301000], 0.255)

(0.744326923076923, 0.3883946757485272, 0.5922999999999999)

In [15]:
rf3p6 = RandomForestClassifier(min_samples_split=6)
rf3p6.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_split=6)

In [16]:
get_metrics(rf3p6, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3p6, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3p6, states32d[300000:301000], hands[300000:301000], 0.25)

((0.7384615384615386, 0.3848972357272482, 0.5952000000000001),
 (0.7333461538461538, 0.3767964718712212, 0.5995),
 (0.7393076923076923, 0.3882503266899939, 0.6256000000000002))

In [38]:
rf3p8 = RandomForestClassifier(min_samples_split=8)
rf3p8.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_split=8)

In [61]:
get_metrics(rf3p8, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3p8, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3p8, states32d[300000:301000], hands[300000:301000], 0.25)

((0.7380769230769231, 0.38485664854581264, 0.5984),
 (0.737423076923077, 0.38479276908386484, 0.6115),
 (0.7377884615384616, 0.38547544905883807, 0.6247999999999999))

In [62]:
get_metrics(rf3p8, states32d[100000:101000], hands[100000:101000], 0.26), \
get_metrics(rf3p8, states32d[200000:201000], hands[200000:201000], 0.26), \
get_metrics(rf3p8, states32d[300000:301000], hands[300000:301000], 0.26)

((0.7528461538461538, 0.39534266111855115, 0.5448999999999999),
 (0.7503846153846154, 0.39275702525029144, 0.559),
 (0.7511346153846155, 0.3936231785586275, 0.57))

In [63]:
get_metrics(rf3p8, states32d[100000:101000], hands[100000:101000], 0.27), \
get_metrics(rf3p8, states32d[200000:201000], hands[200000:201000], 0.27), \
get_metrics(rf3p8, states32d[300000:301000], hands[300000:301000], 0.27)

((0.7652307692307693, 0.4061706362737709, 0.4899),
 (0.762, 0.40037605689374267, 0.5031),
 (0.763826923076923, 0.40439458216791807, 0.5182))

In [17]:
rf3p88 = RandomForestClassifier(min_samples_split=8, min_samples_leaf=4)
rf3p88.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_leaf=4, min_samples_split=8)

In [18]:
# no sample weight
get_metrics(rf3p88, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3p88, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3p88, states32d[300000:301000], hands[300000:301000], 0.25)

((0.7378076923076924, 0.3851137601222606, 0.5943999999999999),
 (0.7345, 0.37782028676517837, 0.601),
 (0.741076923076923, 0.391717926745597, 0.6263))

In [57]:
# sample weight
get_metrics(rf3p88, states32d[100000:101000], hands[100000:101000], 0.26), \
get_metrics(rf3p88, states32d[200000:201000], hands[200000:201000], 0.26), \
get_metrics(rf3p88, states32d[300000:301000], hands[300000:301000], 0.26)

((0.74025, 0.37711397994850004, 0.5595),
 (0.7375961538461538, 0.37451855645644344, 0.5611000000000002),
 (0.7438461538461538, 0.3860332238772355, 0.5852999999999999))

In [75]:
rf3p12 = RandomForestClassifier(min_samples_leaf=6)
rf3p12.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_leaf=6)

In [76]:
get_metrics(rf3p12, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3p12, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3p12, states32d[300000:301000], hands[300000:301000], 0.25)

((0.7365384615384616, 0.3821874702892791, 0.5942),
 (0.7355769230769232, 0.38088011805035127, 0.6056),
 (0.7378846153846154, 0.3848285144997387, 0.6201))

In [34]:
rf3p2 = RandomForestClassifier(min_samples_leaf=8)
rf3p2.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_leaf=8)

In [58]:
get_metrics(rf3p2, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3p2, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3p2, states32d[300000:301000], hands[300000:301000], 0.25)

((0.7372692307692307, 0.3842978596910485, 0.5938),
 (0.7359615384615386, 0.38086428454586346, 0.6047),
 (0.7398076923076924, 0.38896532059272, 0.6246))

In [59]:
get_metrics(rf3p2, states32d[100000:101000], hands[100000:101000], 0.26), \
get_metrics(rf3p2, states32d[200000:201000], hands[200000:201000], 0.26), \
get_metrics(rf3p2, states32d[300000:301000], hands[300000:301000], 0.26)

((0.7527115384615385, 0.3942305356580124, 0.5311),
 (0.7522692307692307, 0.3915325254166198, 0.5458999999999999),
 (0.7536153846153847, 0.39583077844927694, 0.5622))

In [60]:
get_metrics(rf3p2, states32d[100000:101000], hands[100000:101000], 0.27), \
get_metrics(rf3p2, states32d[200000:201000], hands[200000:201000], 0.27), \
get_metrics(rf3p2, states32d[300000:301000], hands[300000:301000], 0.27)

((0.7680576923076924, 0.40955349774494504, 0.46929999999999994),
 (0.7650576923076924, 0.39895463123048264, 0.48229999999999995),
 (0.7662884615384615, 0.403016821547046, 0.4988))

In [30]:
rf3pl4 = RandomForestClassifier(min_samples_leaf=4)
rf3pl4.fit(states32d[:100000], hands[:100000])

RandomForestClassifier(min_samples_leaf=4)

In [109]:
get_metrics(rf3pl4, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rf3pl4, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rf3pl4, states32d[300000:301000], hands[300000:301000], 0.25)

('acc: 0.8738, prec: 0.6710, rec: 0.8824, sds:  0.1152, 0.2284, 0.1678',
 'acc: 0.7196, prec: 0.3760, rec: 0.6649, sds:  0.0763, 0.1089, 0.1826',
 'acc: 0.7250, prec: 0.3847, rec: 0.6855, sds:  0.0781, 0.1102, 0.1909')

In [115]:
get_metrics(rf3pl4, states32d[400000:401000], hands[400000:401000], 0.25), \
get_metrics(rf3pl4, states32d[500000:501000], hands[500000:501000], 0.25), \
get_metrics(rf3pl4, states32d[600000:601000], hands[600000:601000], 0.25)

('acc: 0.7222, prec: 0.3790, rec: 0.6604, sds:  0.0764, 0.1094, 0.1825',
 'acc: 0.7284, prec: 0.3907, rec: 0.6893, sds:  0.0801, 0.1149, 0.1868',
 'acc: 0.7193, prec: 0.3773, rec: 0.6717, sds:  0.0820, 0.1145, 0.1933')

In [48]:
get_metrics(rf3pl4, states32d[100000:101000], hands[100000:101000], 0.25)

(0.7384615384615386, 0.386083877210718, 0.6008000000000001)

In [49]:
get_metrics(rf3pl4, states32d[200000:201000], hands[200000:201000], 0.25)

(0.7326538461538462, 0.3753762261956198, 0.5968000000000001)

In [50]:
get_metrics(rf3pl4, states32d[300000:301000], hands[300000:301000], 0.25)

(0.7391346153846152, 0.38664257091377835, 0.6186)

# only predict for after opponent has done at least 2 moves (sum > 14)

In [15]:
states14 = states32d[sum_states2d > 14]

In [16]:
hands14 = hands[sum_states2d > 14]

In [ ]:
# random sampling for test and train

In [14]:
states14.shape

(1760943, 156)

In [17]:
rf3pl4 = RandomForestClassifier(min_samples_leaf=4)
rf3pl4.fit(states14[:100000], hands14[:100000])

RandomForestClassifier(min_samples_leaf=4)

In [124]:
rf3pl4.feature_importances_.argsort()

array([ 92,  55,  80,  65,  68,  94,  93,  54,  78,  82,  91,  79,  66,
        95,  52,  67,  81,  69,  56,  83,  53,  70,   3,  57,  96,  16,
        28,   2,  13,  15,  42,  29,  27,   1,  14,  40,  41,   0,  39,
        26,  84,  58,  71,  97,  43,  17,  30,   4,  72,  98,  85,  59,
       155, 142, 129,  31, 116,  44,  18, 128,   5, 141, 154,  60, 115,
       103, 127, 140, 153,  73, 139, 114, 102, 126,  76, 113,  86, 152,
        99,  77,   6,  63,  89,  19,  45,  90,  32, 101,  75,  62,  64,
        61, 112, 125, 138,  88, 151,  46,  87, 124,  20, 111,  33,   7,
       137,  74, 100, 150, 136, 123, 149, 110,  47,   8,  34,  21, 135,
       148, 109, 122,  12,  38,  11,  36, 119, 108, 146, 120,  24, 121,
       134, 118,  35,  51,  25,  50, 106, 104, 147,  23,  22,  49,  48,
        37, 117, 143, 107, 145,  10, 132, 144, 131,   9, 133, 105, 130])

In [110]:
get_metrics(rf3pl4, states14[100000:101000], hands14[100000:101000], 0.25), \
get_metrics(rf3pl4, states14[200000:201000], hands14[200000:201000], 0.25), \
get_metrics(rf3pl4, states14[300000:301000], hands14[300000:301000], 0.25)

('acc: 0.7614, prec: 0.4356, rec: 0.7570, sds:  0.0682, 0.1012, 0.1676',
 'acc: 0.7542, prec: 0.4252, rec: 0.7352, sds:  0.0699, 0.1062, 0.1763',
 'acc: 0.7551, prec: 0.4269, rec: 0.7501, sds:  0.0686, 0.0992, 0.1679')

In [121]:
get_metrics(rf3pl4, states14[100000:101000], hands14[100000:101000], 0.27), \
get_metrics(rf3pl4, states14[200000:201000], hands14[200000:201000], 0.27), \
get_metrics(rf3pl4, states14[300000:301000], hands14[300000:301000], 0.27)

('acc: 0.7760, prec: 0.4501, rec: 0.6772, sds:  0.0667, 0.1168, 0.1810',
 'acc: 0.7682, prec: 0.4359, rec: 0.6510, sds:  0.0696, 0.1252, 0.1972',
 'acc: 0.7715, prec: 0.4433, rec: 0.6782, sds:  0.0679, 0.1157, 0.1827')

In [35]:
get_metrics(rf3pl4, states14[100000:101000], hands14[100000:101000], 0.3), \
get_metrics(rf3pl4, states14[200000:201000], hands14[200000:201000], 0.3), \
get_metrics(rf3pl4, states14[300000:301000], hands14[300000:301000], 0.3)

((0.79425, 0.47493049230181583, 0.5362),
 (0.7882115384615385, 0.4593347985347985, 0.5073),
 (0.7893461538461538, 0.46533821276762455, 0.5332))

In [36]:
get_metrics(rf3pl4, states14[100000:101000], hands14[100000:101000], 0.2), \
get_metrics(rf3pl4, states14[200000:201000], hands14[200000:201000], 0.2), \
get_metrics(rf3pl4, states14[300000:301000], hands14[300000:301000], 0.2)

((0.6971346153846154, 0.38096205392066257, 0.8887),
 (0.6906923076923077, 0.3747471016268892, 0.8788),
 (0.68525, 0.36970009942719667, 0.8752000000000001))

In [97]:
predict_cards_t(predict_opponent_hand(rf3pl4, states32d[100000]), 0.3)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [100]:
predict_opponent_hand(rf3pl4, states32d[100000])

array([0.04850275, 0.48865366, 0.55083505, 0.262321  , 0.17529011,
       0.21516099, 0.09137859, 0.12865693, 0.14379256, 0.03651992,
       0.02660087, 0.01784271, 0.02449299, 0.56746169, 0.46233836,
       0.22226285, 0.52502775, 0.19145343, 0.23061444, 0.18999786,
       0.4912247 , 0.59404097, 0.36998132, 0.08799179, 0.06267002,
       0.05130105, 0.25368217, 0.22279219, 0.17398577, 0.50479256,
       0.21776975, 0.14459601, 0.0891109 , 0.06179034, 0.17762392,
       0.04738275, 0.02683613, 0.02083613, 0.03017617, 0.25817038,
       0.57444177, 0.12683795, 0.06882663, 0.12894447, 0.14633802,
       0.11377629, 0.07956027, 0.18878938, 0.01867385, 0.02567374,
       0.01459127, 0.02758682])

In [72]:
sum(states14[400000])

16.0

In [99]:
hands[100000].astype(int)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
with open("rf.obj","wb") as f:
    dill.dump(rf0,f)

In [117]:
states16 = states32d[sum_states2d > 16]

In [118]:
hands16 = hands[sum_states2d > 16]

In [119]:
rf3p16 = RandomForestClassifier(min_samples_leaf=4)
rf3p16.fit(states16[:100000], hands16[:100000])

RandomForestClassifier(min_samples_leaf=4)

In [120]:
get_metrics(rf3p16, states16[100000:101000], hands16[100000:101000], 0.25), \
get_metrics(rf3p16, states16[200000:201000], hands16[200000:201000], 0.25), \
get_metrics(rf3p16, states16[300000:301000], hands16[300000:301000], 0.25)

('acc: 0.7512, prec: 0.4200, rec: 0.7488, sds:  0.0601, 0.0881, 0.1580',
 'acc: 0.7518, prec: 0.4217, rec: 0.7523, sds:  0.0681, 0.0992, 0.1698',
 'acc: 0.7532, prec: 0.4241, rec: 0.7560, sds:  0.0651, 0.0943, 0.1615')

In [29]:
get_metrics_rand(states[sum_states2d > 14][100000:101000], hands14[100000:101000]), \
get_metrics_rand(states[sum_states2d > 14][200000:201000], hands14[200000:201000]), \
get_metrics_rand(states[sum_states2d > 14][300000:301000], hands14[300000:301000])

((0.7506923076923077, 0.3518, 0.3518),
 (0.7528846153846154, 0.3575, 0.3575),
 (0.7528461538461538, 0.3574, 0.3574))

In [103]:
states18 = states32d[sum_states2d > 18]
hands18 = hands[sum_states2d > 18]

In [116]:
states18.shape

(1034019, 156)

In [104]:
rf3p18 = RandomForestClassifier(min_samples_leaf=4)
rf3p18.fit(states18[:100000], hands18[:100000])

RandomForestClassifier(min_samples_leaf=4)

In [112]:
get_metrics(rf3p18, states18[100000:101000], hands18[100000:101000], 0.25), \
get_metrics(rf3p18, states18[200000:201000], hands18[200000:201000], 0.25), \
get_metrics(rf3p18, states18[300000:301000], hands18[300000:301000], 0.25)

('acc: 0.7684, prec: 0.4454, rec: 0.7794, sds:  0.0637, 0.0949, 0.1582',
 'acc: 0.7581, prec: 0.4289, rec: 0.7587, sds:  0.0623, 0.0920, 0.1661',
 'acc: 0.7731, prec: 0.4509, rec: 0.7828, sds:  0.0642, 0.0979, 0.1683')

# weight most recent moves more heavily

In [49]:
states_d = np.load('states_d.npy')
hands_d = np.load('hands_d.npy')

In [9]:
states_d.shape

(1526432, 3, 52)

In [50]:
states_d2d = states_d.reshape(1526432, 3*52)

In [11]:
rfd = RandomForestClassifier(min_samples_leaf=4)
rfd.fit(states_d2d[:100000], hands_d[:100000])

RandomForestClassifier(min_samples_leaf=4)

In [113]:
get_metrics(rfd, states_d2d[100000:101000], hands_d[100000:101000], 0.25), \
get_metrics(rfd, states_d2d[200000:201000], hands_d[200000:201000], 0.25), \
get_metrics(rfd, states_d2d[300000:301000], hands_d[300000:301000], 0.25)

('acc: 0.7370, prec: 0.3853, rec: 0.6203, sds:  0.0696, 0.1180, 0.2193',
 'acc: 0.7292, prec: 0.3721, rec: 0.5924, sds:  0.0689, 0.1165, 0.2113',
 'acc: 0.7377, prec: 0.3845, rec: 0.6157, sds:  0.0676, 0.1136, 0.2149')

In [13]:
get_metrics(rfd, states_d2d[100000:101000], hands_d[100000:101000], 0.26), \
get_metrics(rfd, states_d2d[200000:201000], hands_d[200000:201000], 0.26), \
get_metrics(rfd, states_d2d[300000:301000], hands_d[300000:301000], 0.26)

((0.7498461538461538, 0.39094652485457443, 0.5607000000000001),
 (0.7459615384615385, 0.38441055860204465, 0.5328999999999999),
 (0.7508846153846154, 0.39041708438190637, 0.5553999999999999))

In [55]:
sum_states_d = np.sum(states_d2d, axis=1)

In [17]:
sum_states_d[:20]

array([12.     , 12.     , 13.9    , 13.9    , 15.61   , 15.61   ,
       17.149  , 17.149  , 18.5341 , 18.5341 , 19.88069, 12.     ,
       12.     , 13.8    , 13.9    , 12.     , 12.     , 13.8    ,
       13.9    , 15.42   ])

In [56]:
states_d14 = states_d2d[sum_states_d > 14]
hands_d14 = hands_d[sum_states_d > 14]

In [21]:
rfdl4 = RandomForestClassifier(min_samples_leaf=4)
rfdl4.fit(states_d14[:100000], hands_d14[:100000])

RandomForestClassifier(min_samples_leaf=4)

In [114]:
get_metrics(rfdl4, states_d14[100000:101000], hands_d14[100000:101000], 0.25), \
get_metrics(rfdl4, states_d14[200000:201000], hands_d14[200000:201000], 0.25), \
get_metrics(rfdl4, states_d14[300000:301000], hands_d14[300000:301000], 0.25)

('acc: 0.7413, prec: 0.4062, rec: 0.7158, sds:  0.0693, 0.0992, 0.1719',
 'acc: 0.7479, prec: 0.4159, rec: 0.7256, sds:  0.0709, 0.1053, 0.1767',
 'acc: 0.7457, prec: 0.4122, rec: 0.7297, sds:  0.0686, 0.1008, 0.1748')

In [122]:
get_metrics(rfdl4, states_d14[100000:101000], hands_d14[100000:101000], 0.27), \
get_metrics(rfdl4, states_d14[200000:201000], hands_d14[200000:201000], 0.27), \
get_metrics(rfdl4, states_d14[300000:301000], hands_d14[300000:301000], 0.27)

('acc: 0.7573, prec: 0.4175, rec: 0.6305, sds:  0.0675, 0.1154, 0.1813',
 'acc: 0.7665, prec: 0.4351, rec: 0.6480, sds:  0.0701, 0.1237, 0.1872',
 'acc: 0.7612, prec: 0.4243, rec: 0.6504, sds:  0.0672, 0.1159, 0.1870')

In [102]:
get_metrics(rfdl4, states_d14[400000:401000], hands_d14[400000:401000], 0.25)

((0.7391346153846154, 0.40303304790623745, 0.7177),)

In [77]:
predict_cards_t(predict_opponent_hand(rfdl4, states_d14[400000]), 0.3)

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
predict_opponent_hand(rfdl4, states_d14[400007])

array([0.11695514, 0.13020837, 0.41477651, 0.31721578, 0.32114458,
       0.23503989, 0.15424226, 0.21701587, 0.12759285, 0.06626575,
       0.0483377 , 0.04682913, 0.04927681, 0.38910322, 0.32246562,
       0.26276274, 0.29943815, 0.30840264, 0.17257256, 0.2024778 ,
       0.23270644, 0.1720206 , 0.06565081, 0.06839328, 0.06196805,
       0.06406675, 0.32519415, 0.31179317, 0.38356284, 0.32153378,
       0.25848511, 0.27521643, 0.18748377, 0.32518914, 0.16045403,
       0.07244994, 0.07867174, 0.07021962, 0.0822298 , 0.16602312,
       0.3493189 , 0.30828754, 0.14986694, 0.25186885, 0.29420082,
       0.25056537, 0.21069618, 0.12407223, 0.04764948, 0.03372339,
       0.04573486, 0.04857953])

In [78]:
hands_d14[400000].astype(int)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

# randomize training and test data 

In [20]:
len(states32d)

2571176

In [26]:
inds2 = sample(range(len(states32d)), 110000)

In [27]:
train_inds2 = inds2[:100000]
test_inds2 = inds2[100000:]

In [23]:
rfr2 = RandomForestClassifier(min_samples_leaf=4)
rfr2.fit(states32d[train_inds2], hands[train_inds2])

RandomForestClassifier(min_samples_leaf=4)

In [26]:
get_metrics(rfr2, states32d[test_inds2], hands[test_inds2], 0.25)

'acc: 0.8899, prec: 0.6971, rec: 0.8736, sds:  0.0965, 0.2160, 0.1819'

In [28]:
get_metrics(rfr2, states32d[test_inds2], hands[test_inds2], 0.26)

'acc: 0.9025, prec: 0.7288, rec: 0.8548, sds:  0.0909, 0.2169, 0.2017'

In [30]:
get_metrics(rfr2, states32d[test_inds2], hands[test_inds2], 0.28)

'acc: 0.9195, prec: 0.7844, rec: 0.8124, sds:  0.0817, 0.2202, 0.2426'

In [27]:
get_metrics(rfr2, states32d[100000:100100], hands[100000:100100], 0.25)

'acc: 0.7675, prec: 0.4365, rec: 0.6250, sds:  0.0674, 0.1302, 0.1780'

In [35]:
get_metrics(rfr2, states14[test_inds], hands14[test_inds], 0.25)

'acc: 0.9043, prec: 0.7255, rec: 0.9226, sds:  0.0873, 0.1995, 0.1334'

In [43]:
rfr8 = RandomForestClassifier(min_samples_leaf=2)
rfr8.fit(states32d[train_inds2], hands[train_inds2])

RandomForestClassifier(min_samples_leaf=2)

In [44]:
get_metrics(rfr8, states32d[test_inds2], hands[test_inds2], 0.25)

'acc: 0.9226, prec: 0.7844, rec: 0.9073, sds:  0.0963, 0.2221, 0.1683'

In [57]:
get_metrics(rfr8, states32d[test_inds2], hands[test_inds2], 0.26)

'acc: 0.9296, prec: 0.8015, rec: 0.8948, sds:  0.0906, 0.2186, 0.1869'

In [58]:
get_metrics(rfr8, states32d[test_inds2], hands[test_inds2], 0.28)

'acc: 0.9398, prec: 0.8317, rec: 0.8677, sds:  0.0813, 0.2137, 0.2242'

In [59]:
get_metrics(rfr8, states32d[100000:100100], hands[100000:100100], 0.25)

'acc: 0.7679, prec: 0.4414, rec: 0.6470, sds:  0.0764, 0.1390, 0.1873'

In [63]:
dill_save(rfr8, 'rf2.obj')

In [60]:
rfr1 = RandomForestClassifier()
rfr1.fit(states32d[train_inds2], hands[train_inds2])

RandomForestClassifier()

In [61]:
get_metrics(rfr1, states32d[test_inds2], hands[test_inds2], 0.25)

'acc: 0.9432, prec: 0.8476, rec: 0.9139, sds:  0.0918, 0.2222, 0.1708'

In [62]:
get_metrics(rfr1, states32d[100000:100100], hands[100000:100100], 0.25)

'acc: 0.7735, prec: 0.4462, rec: 0.5930, sds:  0.0855, 0.1882, 0.2141'

In [127]:
len(states14)

1760943

In [18]:
inds = sample(range(len(states14)), 110000)

In [19]:
train_inds = inds[:100000]
test_inds = inds[100000:]

In [20]:
get_metrics(rf3pl4, states14[test_inds], hands14[test_inds], 0.25)

'acc: 0.7576, prec: 0.4353, rec: 0.7425, sds:  0.0803, 0.1327, 0.1766'

In [21]:
rfr14 = RandomForestClassifier(min_samples_leaf=4)
rfr14.fit(states14[train_inds], hands14[train_inds])

RandomForestClassifier(min_samples_leaf=4)

In [22]:
get_metrics(rfr14, states14[test_inds], hands14[test_inds], 0.25)

'acc: 0.9092, prec: 0.7408, rec: 0.9343, sds:  0.0914, 0.2038, 0.1226'

In [43]:
get_metrics(rfr14, states14[test_inds], hands14[test_inds], 0.26)

'acc: 0.9184, prec: 0.7644, rec: 0.9252, sds:  0.0878, 0.2020, 0.1341'

In [34]:
get_metrics(rfr14, states32d[test_inds], hands[test_inds], 0.25)

'acc: 0.8505, prec: 0.6288, rec: 0.8471, sds:  0.1246, 0.2462, 0.1907'

In [25]:
get_metrics(rfr14, states14[100000:101000], hands14[100000:101000], 0.25), \
get_metrics(rfr14, states14[200000:201000], hands14[200000:201000], 0.25), \
get_metrics(rfr14, states14[300000:301000], hands14[300000:301000], 0.25)

('acc: 0.7756, prec: 0.4584, rec: 0.7875, sds:  0.0725, 0.1127, 0.1674',
 'acc: 0.9381, prec: 0.8069, rec: 0.9636, sds:  0.0718, 0.1746, 0.0859',
 'acc: 0.9295, prec: 0.7839, rec: 0.9575, sds:  0.0754, 0.1790, 0.0926')

In [26]:
get_metrics(rfr14, states32d[100000:101000], hands[100000:101000], 0.25), \
get_metrics(rfr14, states32d[200000:201000], hands[200000:201000], 0.25), \
get_metrics(rfr14, states32d[300000:301000], hands[300000:301000], 0.25)

('acc: 0.7316, prec: 0.3944, rec: 0.6844, sds:  0.0823, 0.1204, 0.1907',
 'acc: 0.7362, prec: 0.4007, rec: 0.6962, sds:  0.0815, 0.1207, 0.1901',
 'acc: 0.8826, prec: 0.6944, rec: 0.8826, sds:  0.1167, 0.2382, 0.1757')

In [46]:
rfr14.feature_importances_.argsort()

array([ 79,  66,  65,  92,  93,  55,  52,  94,  81,  91,  69,  53,  80,
        67,  54,  95,  82,  78,  56,  57,   2,  68,  70,  83,  96,  29,
         0,  41,  27,  13,  26,   3,  28,  15,  40,  16,  14,   1,  42,
        58,  39,  97,  84,  71,  30,   4,  17,  43,  98,  85,  59,  72,
       155, 142,  86,  31,  73, 116, 129,  18,  44,   5,  60, 128, 141,
       154,  64, 115,  77,  99, 114, 113, 140, 153, 127, 126, 103,  90,
       152, 139, 102,  76, 101,  62,  89,  45,  19,  88,  63,  32,   6,
        87,  75,  74, 100, 151, 112, 138, 125,  61,  46,   7,  33,  20,
       111, 137, 124, 150, 110, 123, 136, 149,  47,  21,   8,  34, 148,
       135, 109, 122, 104, 106, 145, 143, 119, 146, 117, 144, 118, 134,
        12,  37, 147, 108, 120, 132, 107, 121,  38,  25, 133,  36, 130,
        51,  23,  50,  49, 131,  22,  11,  10,  35,   9, 105,  24,  48])

In [48]:
rfr14s = RandomForestClassifier(min_samples_leaf=4, min_samples_split=4)
rfr14s.fit(states14[train_inds], hands14[train_inds])

RandomForestClassifier(min_samples_leaf=4, min_samples_split=4)

In [51]:
get_metrics(rfr14s, states14[test_inds], hands14[test_inds], 0.25)

'acc: 0.9088, prec: 0.7404, rec: 0.9332, sds:  0.0919, 0.2051, 0.1235'

In [58]:
inds3 = sample(range(len(states_d14)), 110000)
train_inds3 = inds3[:100000]
test_inds3 = inds3[100000:]

In [59]:
rfrd14 = RandomForestClassifier(min_samples_leaf=4)
rfrd14.fit(states_d14[train_inds3], hands_d14[train_inds3])

RandomForestClassifier(min_samples_leaf=4)

In [60]:
get_metrics(rfrd14, states_d14[test_inds3], hands_d14[test_inds3], 0.25)

'acc: 0.8147, prec: 0.5328, rec: 0.8539, sds:  0.0909, 0.1638, 0.1627'

In [ ]:
# how often does opponent pick from discard pile?

In [62]:
states32d[sum_states2d > 18][:,52:104].shape

(1760943, 52)

In [69]:
temp = states32d[sum_states2d > 20][:,52:104].sum(axis=1)

In [72]:
np.mean(temp), np.median(temp), np.std(temp)

(0.8331296067421066, 1.0, 0.8493636545204086)

# Directly predict what's in the draw pile

In [73]:
hands.shape

(2571176, 52)

In [20]:
pile = np.zeros((2571176, 52))

In [21]:
known = states32d[:,:52] + states32d[:,52:104] + states32d[:,104:] + hands

In [22]:
known[known > 1] = 1

In [23]:
pile = 1 - known

In [40]:
rfpile = RandomForestClassifier(min_samples_leaf=4)
rfpile.fit(states32d[train_inds2], pile[train_inds2])

RandomForestClassifier(min_samples_leaf=4)

In [41]:
predict_opponent_hand(rfpile, states32d[102000])

array([0.33746565, 0.54961626, 0.48969099, 0.59257044, 0.35065252,
       0.38682153, 0.42422748, 0.43479037, 0.49892854, 0.60086149,
       0.46901007, 0.63937902, 0.64903898, 0.53656733, 0.51482935,
       0.49417144, 0.46260994, 0.48910171, 0.41945893, 0.54474525,
       0.58862513, 0.6164573 , 0.19959224, 0.68780603, 0.68107837,
       0.17799965, 0.54410037, 0.57216117, 0.3632502 , 0.3407094 ,
       0.44109234, 0.5451472 , 0.57564063, 0.47871375, 0.43911355,
       0.32346161, 0.29076836, 0.22656658, 0.71270452, 0.31068026,
       0.54948486, 0.53390004, 0.40582029, 0.51751168, 0.56449439,
       0.57582776, 0.57737602, 0.54482471, 0.72262143, 0.19479898,
       0.42678113, 0.73830057])

In [51]:
get_metrics(rfpile, states32d[test_inds2], pile[test_inds2], 0.42)

'acc: 0.8452, prec: 0.7955, rec: 0.9751, sds:  0.1147, 0.1389, 0.0943'

In [52]:
get_metrics(rfpile, states32d[test_inds2], pile[test_inds2], 0.45)

'acc: 0.8750, prec: 0.8394, rec: 0.9559, sds:  0.1030, 0.1308, 0.1273'

In [56]:
dill_save(rfpile, 'rfpile.obj')

In [28]:
rfpile2 = RandomForestClassifier(min_samples_leaf=2)
rfpile2.fit(states32d[train_inds2], pile[train_inds2])

RandomForestClassifier(min_samples_leaf=2)

In [72]:
get_metrics(rfpile2, states32d[test_inds2], pile[test_inds2], 0.45)

'acc: 0.9137, prec: 0.8875, rec: 0.9675, sds:  0.1012, 0.1288, 0.1043'

In [29]:
get_metrics(rfpile2, states32d[test_inds2], pile[test_inds2], 0.42)

/Users/mayjiang/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  


'acc: 0.8914, prec: nan, rec: 0.9796, sds:  0.1156, nan, 0.0795'

# Predicting opponent's next discard

In [2]:
# given opponent's actual hand, opponent discards, predict which one opponent discards next 
# --> then feed it predicted opponent hand
# OR, directly predict given your hand, opponent discards, opponent known cards, what the opponent discards next

In [35]:
def un_one_hot(arr):
    rankNames = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K"]
    suitNames = ["C", "H", "S", "D"]
    ret = []
    for i in range(len(arr)):
        if arr[i] != 0:
            ret.append(rankNames[i%13] + suitNames[i//13])
    return ret

In [55]:
states_dis = np.load('states_dis.npy')
hands_dis = np.load('hands_dis.npy')
dis = np.load('dis.npy')

In [56]:
for x in states_dis[:5]:
    for y in x:
        print(un_one_hot(y))
    print("-----")

['QS']
['JH']
['3H', '6H', 'QH', '6S', '7S', 'AD', '2D', '6D', '7D', '9D']
-----
['KC', 'QS']
[]
['5C', 'AH', '2H', 'JH', '3S', '4S', 'JS', '4D', '5D', 'JD']
-----
['6C', 'KC', 'QS']
['JH']
['3H', '6H', 'QH', '6S', '7S', 'AD', '2D', '6D', '7D', '9D']
-----
['KC', 'QH', 'QS']
['6C']
['5C', 'AH', '2H', 'JH', '3S', '4S', 'JS', '4D', '5D', 'JD']
-----
['6C', 'KC', 'TH', 'QH', 'QS']
['JH']
['6C', '3H', '6H', '6S', '7S', 'AD', '2D', '6D', '7D', '9D']
-----


In [57]:
for x in hands_dis[:5]:
    print(un_one_hot(x))

['5C', 'AH', '2H', 'JH', '3S', '4S', 'JS', '4D', '5D', 'JD']
['3H', '6H', 'QH', '6S', '7S', 'AD', '2D', '6D', '7D', '9D']
['5C', 'AH', '2H', 'JH', '3S', '4S', 'JS', '4D', '5D', 'JD']
['6C', '3H', '6H', '6S', '7S', 'AD', '2D', '6D', '7D', '9D']
['5C', 'AH', '2H', 'JH', '3S', '4S', 'JS', '4D', '5D', 'JD']


In [58]:
for x in dis[:5]:
    print(un_one_hot(x))

['QS']
['KC']
['6C']
['QH']
['TH']


In [59]:
hands_dis2 = hands_dis[:-2]
dis2 = dis[2:]
hands_dis2.shape, dis2.shape

((1271934, 52), (1271934, 52))

In [63]:
for x in dis2[:5]:
    print(un_one_hot(x))

['6C']
['QH']
['TH']
['JC']
['5D']


In [66]:
inds = sample(range(len(hands_dis2)), 200000)
train_inds = inds[:100000]
test_inds = inds[100000:]

In [67]:
rfdis = RandomForestClassifier(min_samples_leaf=2)
rfdis.fit(hands_dis2[train_inds], dis2[train_inds])

RandomForestClassifier(min_samples_leaf=2)

In [73]:
np.argmin([x[0][0] for x in rfdis.predict_proba([hands_dis2[100000]])])

24

In [82]:
correct = 0
for i, x in enumerate(hands_dis2[test_inds[:100]]):
    if np.argmax([y[0][1] for y in rfdis.predict_proba([x])]) == np.argmax(dis2[i]):
        correct += 1
correct/100

0.05